# Falsk App POC
- The purpose of htis  notebook is to perform POC to deploy a model on on Flask server , which runs locally on the machine.
- Once this works, the same script will be made into app.py and deploy it on to AWS EC2 server

- Below cell reads the logisticRegressionModel.pkl file, which is a trained ST16 Model LR file.We use this model to make prediction 
- Once the below cell is run, the app will be serving on localhost:5000 port
- Afer succesfull execution of the below cell, we can test this by pasting below URL in the browser http://127.0.0.1:5000/predict/?rainFall=20.4&siteId=2
- Here siteId, is the used to identiy a particular record from ST16 data and the rainFall will be used as i15 parameter
- we can change the 2 parameters to test the predict function

In [ ]:
from flask import Flask,request
import pandas as pd
import numpy as np
import pickle

app = Flask(__name__)

#read data into pandas df
sdf = pd.read_csv("../../data/ofr20161106-na-omit.csv")
# add ID column , which can be used to identity a paritculay record
sdf['ID'] = np.arange(sdf.shape[0])

#load ST16 LR Model from pickle file 
with open("../../data/ST16_LogisticRegressionModel.pkl",'rb') as file:
    lr_model = pickle.load(file)

#URL Binding     
@app.route('/predict/', methods=['GET'])
def predict():
    # read parameters from the URL
    parameters = request.args.to_dict()
    i15 = float(parameters['rainFall'])
    siteId = int(parameters['siteId'])
    
    #identify the site's record
    rec = sdf[sdf['ID']==siteId][["PropHM23","dNBR/1000",'KF']]
    
    # multiply the peak rainfall intensity ( value is derived from parameters)
    rec["PropHM23_x_i15"] = rec["PropHM23"] * i15
    rec["dNBR_x_i15"] = rec["dNBR/1000"] * i15
    rec["KF_x_i15"] = rec["KF"] * i15

    #select the features which are required for model prediction
    rec = rec[["PropHM23_x_i15","dNBR_x_i15","KF_x_i15"]]

    #predict
    y_test_pred = lr_model.predict(rec)   
    #return the output
    return 'Chance of having debris flow : %s' %y_test_pred[0]
                 
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/May/2023 19:16:07] "GET /predict/?rainFall=20.4&siteId=2 HTTP/1.1" 200 -


- Below is code to read the model from NN model.pth and use to make predictions ( not tested)

In [25]:
seed=27
import numpy as np
import pandas as pd
import geopandas as gpd
import sklearn
import os

# # visualization libraries
from matplotlib import pyplot as plt
# plt.style.use('bmh')
plt.style.use('dark_background')
# plt.style.use('fivethirtyeight')
# Have plots display in notebook
%matplotlib inline
# import seaborn as sns

# # ML libraries
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, RocCurveDisplay#, roc_curve
# from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

# # for sigmoid function, in case we need to manually implement in LR
# from scipy.stats import logistic

import torch
import torch.nn as nn
import torch.optim as optim
import pickle

In [8]:
#read data
df = modelDataI=gpd.read_parquet("../../data/staley16_observations_catchment_fuelpars_rocktype_randn_v3.parquet")
df=df.dropna()
df.shape
features_15 = [
    'peak_i15_mmh', 
    'contributingarea_km2', 
    'prophm23',
    'dnbr1000', 
    'kf', 
    'acc015_mm', 
    'NB', 
    'GR', 
    'GS', 
    'SH', 
    'TU', 
    'TL',
    'Fine fuel load', 
    'SAV', 
    'Packing ratio', 
    'Extinction moisture content',
    'Igneous', 
    'Metamorphic', 
    'Sedimentary', 
    'Unconsolidated']

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate) # dropout to prevent overfitting
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate) # dropout to prevent overfitting
        
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_rate) # dropout to prevent overfitting

        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_rate) # dropout to prevent overfitting
        
        self.fc5 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid() # output to probability rather than bool
        
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        
        out = self.fc5(out)
        out = self.sigmoid(out)      

        return out

In [36]:
if __name__ == '__main__':
    # Load data
    data = df.copy()
    
    X = data[features_15]
    y = data['response']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    #scale the data X_train and X_test
    cols = X_train.columns
    sc = StandardScaler()
    X_train = pd.DataFrame(sc.fit_transform(X_train), columns=cols)
    X_test = pd.DataFrame(sc.transform(X_test), columns=cols)
    
    X_train = torch.tensor(X_train.values).float()
    y_train = torch.tensor(y_train.values).float().view(-1, 1)
    X_test = torch.tensor(X_test.values).float()
    y_test = torch.tensor(y_test.values).float().view(-1, 1)
    
    input_size = X_train.shape[1]
    hidden_size = 500
    output_size = 1
    dropout_rate = 0.2
    learning_rate = 0.001 # 0.001 is default value for Adam optimizer
    
    model = Net(input_size, hidden_size, output_size, dropout_rate)
    model.load_state_dict(torch.load('model.pth'))
#     criterion = nn.BCELoss()  # cross-entropy better suited for binary classification than MSE
 
    
#     #criterion = nn.BCEWithLogitsLoss(pos_weight=torch.ones([output_size]))
#     #criterion = nn.NLLLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      
#     num_epochs = 500
    
#     # empty df to track loss over epochs
#     loss_df = pd.DataFrame(columns=['train_loss', 'test_loss'])
    
#     for epoch in range(num_epochs):
#         y_train_pred = model(X_train)
#         loss = criterion(y_train_pred, y_train)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         if epoch%5 ==0:
#             y_train_pred = model(X_train)
#             y_test_pred = model(X_test)

#             train_loss = criterion(y_train_pred, y_train)
#             val_loss = criterion(y_test_pred, y_test)
            
#             loss_df.loc[epoch, 'train_loss'] = train_loss.detach().numpy().reshape(1)[0]
#             loss_df.loc[epoch, 'test_loss'] = val_loss.detach().numpy().reshape(1)[0]

# fig = loss_df.plot()
# fig.set_xlabel('epochs')
# fig.set_ylabel('loss')
# fig.set_title('Loss over Epochs');

# now final outputs
y_train_pred = model(X_train)
#y_train_prob = torch.sigmoid(y_train_pred) # already a probability

y_test_pred = model(X_test)
# y_test_prob = torch.sigmoid(y_test_pred) # already a probability


# print("\n")

# # accuracy
# train_pred_correct = sum(y_train.detach().numpy() == np.round(y_train_pred.detach().numpy()))
# train_accuracy = (train_pred_correct / y_train.shape[0])[0]
# print(f'Training accuracy: {train_accuracy}')

# test_pred_correct = sum(y_test.detach().numpy() == np.round(y_test_pred.detach().numpy()))
# test_accuracy = (test_pred_correct / y_test.shape[0])[0]
# print(f'Test accuracy: {test_accuracy}')

# # f1
f1_output = f1_score(y_test, np.round(y_test_pred.detach().numpy()))
print("\n")
print(f'F1 Score (test): {f1_output}')


# # extract AUC for printing
auc_test = roc_auc_score(
    y_test.detach().numpy(), 
    y_test_pred.detach().numpy()
)
print(f'AUC (test): {auc_test}')
print('\n')

# # and plot AUC
# fig, ax = plt.subplots()

# RocCurveDisplay.from_predictions(y_train.detach().numpy(), 
#                                  y_train_pred.detach().numpy(), 
#                                  ax=ax, 
#                                  linewidth=1, 
#                                  color='red', 
#                                  name='train', 
#                                  linestyle="dashed"
#                                 );

# RocCurveDisplay.from_predictions(y_test.detach().numpy(), 
#                                  y_test_pred.detach().numpy(),
#                                  ax=ax, 
#                                  linewidth=1, 
#                                  color='yellow', 
#                                  name='test');

# plt.title('FC NN w/ BCELoss: 15 min rain accum.');



F1 Score (test): 0.6486486486486486
AUC (test): 0.8543739279588336




In [10]:
import requests
requests.__version__

'2.30.0'

In [115]:
sdf = pd.read_csv("../../data/ofr20161106-na-omit.csv")
sdf['ID'] = np.arange(sdf.shape[0])

def test():
    parameters = {'rainFall':'20.4','siteId':'1'}
    i15 = float(parameters['rainFall'])
    siteId = int(parameters['siteId'])
    print(f'Site {0} has {1} rainfall'.format(i15,siteId))
    rec = sdf[sdf['ID']==1][["PropHM23","dNBR/1000",'KF']]
    print(rec)
    rec["PropHM23_x_i15"] = rec["PropHM23"] * i15
    rec["dNBR_x_i15"] = rec["dNBR/1000"] * i15
    rec["KF_x_i15"] = rec["KF"] * i15
    rec = rec[["PropHM23_x_i15","dNBR_x_i15","KF_x_i15"]]
    print(rec[["PropHM23_x_i15","dNBR_x_i15","KF_x_i15"]])
    y_test_pred = lr_model.predict(rec)      
    return 'Chance of having debris flow : %s' %y_test_pred[0]           

print(test())

Site 0 has 1 rainfall
   PropHM23  dNBR/1000    KF
1  0.061249   0.224896  0.25
   PropHM23_x_i15  dNBR_x_i15  KF_x_i15
1        1.249474     4.58787       5.1
Chance of having debris flow : 1


In [61]:
sdf['ID']

0          0
1          1
2          2
3          3
4          4
        ... 
1086    1086
1087    1087
1088    1088
1089    1089
1090    1090
Name: ID, Length: 1091, dtype: int64

In [63]:
type(sdf['ID']==1)

pandas.core.series.Series

In [102]:
sdf = pd.read_csv("../../data/ofr20161106-na-omit.csv")
sdf['ID'] = np.arange(sdf.shape[0])
i15=20

rec = sdf[sdf['ID']==1][["PropHM23","dNBR/1000",'KF']]
print(rec)

rec["PropHM23_x_i15"] = rec["PropHM23"] * i15
rec["dNBR_x_i15"] = rec["dNBR/1000"] * i15
rec["KF_x_i15"] = rec["KF"] * i15

rec = rec[["PropHM23_x_i15","dNBR_x_i15","KF_x_i15"]]

print(rec[["PropHM23_x_i15","dNBR_x_i15","KF_x_i15"]])

y_test_pred = lr_model.predict(rec)

print(y_test_pred)

   PropHM23  dNBR/1000    KF
1  0.061249   0.224896  0.25
   PropHM23_x_i15  dNBR_x_i15  KF_x_i15
1        1.224975    4.497912       5.0
[1]
